In [1]:
import time
import os
import subprocess
import sys
import re
import argparse
import collections
import gzip
import math
import shutil



import matplotlib.pyplot as plt
import wandb
import numpy as np
import time
from datetime import datetime
import random

import seaborn as sns
%matplotlib inline
import logging
from silence_tensorflow import silence_tensorflow
#silence_tensorflow()
#os.environ['TPU_LOAD_LIBRARY']='0'
os.environ['TF_ENABLE_EAGER_CLIENT_STREAMING_ENQUEUE']='False'
import tensorflow as tf


import tensorflow.experimental.numpy as tnp
import tensorflow_addons as tfa
from tensorflow import strings as tfs
from tensorflow.keras import mixed_precision
from scipy.stats.stats import pearsonr  
from scipy.stats.stats import spearmanr  
## custom modules
import metrics as metrics
from optimizers import *
import schedulers as schedulers

import training_utils as training_utils

import enformer_performer as enformer_performer

from scipy import stats


2023-03-10 16:15:51.130436: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 16:15:51.291319: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-10 16:15:51.291349: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-10 16:15:52.167023: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='node-2')
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

with strategy.scope():
    options = tf.data.Options()
    options.experimental_distribute.auto_shard_policy=\
        tf.data.experimental.AutoShardPolicy.FILE
    options.deterministic=False
    options.experimental_threading.max_intra_op_parallelism=1
    mixed_precision.set_global_policy('mixed_bfloat16')
    #options.num_devices = 64

    BATCH_SIZE_PER_REPLICA = 1 # batch size 24, use LR ~ 2.5 e -04
    NUM_REPLICAS = strategy.num_replicas_in_sync
    GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * NUM_REPLICAS
    

2023-03-10 16:15:53.506159: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-10 16:15:53.506212: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-10 16:15:53.506245: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tpu-genformer-v2-6): /proc/driver/nvidia/version does not exist


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


2023-03-10 16:15:53.780159: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 16:15:53.803118: I tensorflow/core/distributed_runtime/rpc/grpc_server_lib.cc:447] Started server with target: grpc://localhost:57833
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: node-2


INFO:tensorflow:Initializing the TPU system: node-2


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [3]:
iterators={'human': ('gs://genformer_data/expanded_originals/196k',5313),
           'mouse': ('gs://genformer_data/expanded_originals/196k',1643)}
g = tf.random.Generator.from_non_deterministic_state()
tr_data_it_dict,val_data_it_dict,val_data_TSS_it =  \
    training_utils.return_distributed_iterators(iterators,
                                                "gs://genformer_data/expanded_originals/196k/human/tfrecords_tss",
                                                5313,
                                                 GLOBAL_BATCH_SIZE,
                                                 196608,
                                                 1536,
                                                 320,
                                                 10,
                                                 4,
                                                 10,
                                                 strategy,
                                                 options,
                                                 g)


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [31]:
with strategy.scope():
    inits=training_utils.get_initializers("gs://picard-testing-176520/sonnet_weights/sonnet_weights")
    model = enformer_performer.enformer_performer(
        num_transformer_layers=6,
        num_heads=8,
        heads_channels= {'human': 5313},
        out_length=1536,
        target_length=896,
        stable_variant=True,
        dim=192,
        d_model=1536,
        norm=True,
        max_seq_length=1536,
        nb_random_features=256,
        hidden_size=1536,
        numerical_stabilizer=0.001,
        rel_pos_bins=1536,
        use_mask_pos=False,
        use_rot_emb=True,
        load_init=True,
        inits=inits,
        kernel_transformation="softmax_kernel_transformation",
        normalize=True,
        seed=5)

In [32]:
with strategy.scope():        

    optimizer1 = tf.keras.optimizers.Adam(learning_rate=1.0e-04)

    optimizer2 = tf.keras.optimizers.Adam(learning_rate=1.0e-04)
    optimizers_in = optimizer1,optimizer2

    metric_dict = {}
    organism_dict = {'human': 50}

                               
    dist_train_step, dist_val_step_h,dist_val_step_m, val_step_TSS, build_step,metric_dict = training_utils.return_train_val_functions(model,
                                                                                                                                       50,
                                                                                                                                       organism_dict,
                                                                                                                                       50,
                                                                                                                                       50,
                                                                                                                                       50,
                                                                                                                                       optimizers_in,
                                                                                                                                        4675,
                                                                                                                                         strategy,
                                                                                                                                         metric_dict, 
                                                                                                                                         GLOBAL_BATCH_SIZE,
                                                                                                                                        5.0)

In [33]:
with strategy.scope():        
    ### main training loop
    global_step = 0
    val_losses = []
    val_pearsons = []
    val_R2 = []
    patience_counter = 0
    stop_criteria = False
    best_epoch = 0

    for epoch_i in range(1, 4):
        print('starting epoch_', str(epoch_i))
        start = time.time()
        if epoch_i == 1:
            # run once to build the model w/o updating anything
            build_step(val_data_it_dict['human'])
        break
        assert len(organism_dict.keys()) == len(tr_data_it_dict.keys())
            
        iters = (tr_data_it_dict['human'],
                        tr_data_it_dict['mouse'])
        dist_train_step(iters)
        


        end = time.time()
        duration = (end - start) / 60.
        print('completed epoch ' + str(epoch_i))
        print('hg_train_loss: ' + str(metric_dict['human_tr'].result().numpy()))
        
        print('training duration(mins): ' + str(duration))

        start = time.time()
        dist_val_step_h(val_data_it_dict['human'])

        print('val_loss: ' + str(metric_dict['human_val'].result().numpy()))
        val_losses.append(metric_dict['human_val'].result().numpy())

        print('human_pearsonsR: ')
        pearsonsR=metric_dict['human_pearsonsR'].result()['PearsonR'].numpy()
        print(pearsonsR)

        val_pearsons.append(np.nanmedian(pearsonsR))
        print('human_R2: ')
        print(metric_dict['human_R2'].result()['R2'].numpy())


        end = time.time()
        duration = (end - start) / 60.
        print('completed epoch ' + str(epoch_i) + ' validation')
        print('validation duration(mins): ' + str(duration))
        print('patience counter at: ' + str(patience_counter))

        
        val_step_TSS(val_data_TSS_it)
        

        y_trues = metric_dict['hg_corr_stats'].result()['y_trues'].numpy()
        y_preds = metric_dict['hg_corr_stats'].result()['y_preds'].numpy()
        cell_types = metric_dict['hg_corr_stats'].result()['cell_types'].numpy()
        gene_map = metric_dict['hg_corr_stats'].result()['gene_map'].numpy()
        
        
        
        if (epoch_i > 2):
            stop_criteria,patience_counter,best_epoch = \
                training_utils.early_stopping(current_val_loss=val_losses[-1],
                                                logged_val_losses=val_losses,
                                                current_pearsons=val_pearsons[-1],
                                                logged_pearsons=val_pearsons,
                                                current_epoch=epoch_i,
                                                best_epoch=best_epoch,
                                                save_freq=5,
                                                patience=5,
                                                patience_counter=patience_counter,
                                                min_delta=1.0e-05,
                                                model=enformer_model,
                                                save_directory="gs://picard-testing-176520/test",
                                                saved_model_basename="test_model",
                                                checkpoint=checkpoint)
        #plt.close('all')
        print('patience counter at: ' + str(patience_counter))
        for key, item in metric_dict.items():
            item.reset_state()

starting epoch_ 1


In [37]:
checkpoint_path="gs://picard-testing-176520/sonnet_weights/sonnet_weights"
inside_checkpoint=tf.train.list_variables(tf.train.latest_checkpoint(checkpoint_path))
reader = tf.train.load_checkpoint(checkpoint_path)

In [47]:
model.layers[3].layers[5].layers[0].layers[0].weights

[TPUDistributedVariable:{
   0: <tf.Variable 'sync_batch_norm_fp32/sync_batch_normalization_25/gamma:0' shape=(1280,) dtype=float32, numpy=
 array([1.1897506 , 1.1832818 , 1.0432662 , ..., 0.8558862 , 0.9469875 ,
        0.97384024], dtype=float32)>,
   1: <tf.Variable 'sync_batch_norm_fp32/sync_batch_normalization_25/gamma/replica_1:0' shape=(1280,) dtype=float32, numpy=
 array([1.1897506 , 1.1832818 , 1.0432662 , ..., 0.8558862 , 0.9469875 ,
        0.97384024], dtype=float32)>,
   2: <tf.Variable 'sync_batch_norm_fp32/sync_batch_normalization_25/gamma/replica_2:0' shape=(1280,) dtype=float32, numpy=
 array([1.1897506 , 1.1832818 , 1.0432662 , ..., 0.8558862 , 0.9469875 ,
        0.97384024], dtype=float32)>,
   3: <tf.Variable 'sync_batch_norm_fp32/sync_batch_normalization_25/gamma/replica_3:0' shape=(1280,) dtype=float32, numpy=
 array([1.1897506 , 1.1832818 , 1.0432662 , ..., 0.8558862 , 0.9469875 ,
        0.97384024], dtype=float32)>,
   4: <tf.Variable 'sync_batch_norm_fp32/syn

In [7]:

def deserialize_val_TSS(serialized_example,
                        input_length, output_length,crop_size,
                        max_shift,num_targets):
    """Deserialize bytes stored in TFRecordFile."""
    feature_map = {
        'sequence': tf.io.FixedLenFeature([], tf.string),
        'target': tf.io.FixedLenFeature([], tf.string),
        'tss_mask': tf.io.FixedLenFeature([], tf.string),
        'gene_name': tf.io.FixedLenFeature([], tf.string),
        'bin_unique': tf.io.FixedLenFeature([], tf.string)
    }
    
    shift = 5
    input_seq_length = input_length + max_shift
    interval_end = input_length + shift
    

    example = tf.io.parse_example(serialized_example, feature_map)
    sequence = tf.io.decode_raw(example['sequence'], tf.bool)
    sequence = tf.reshape(sequence, (input_length + max_shift, 4))
    sequence = tf.cast(sequence, tf.float32)
    sequence = tf.slice(sequence, [shift,0],[input_length,-1])
    
    target = tf.io.decode_raw(example['target'], tf.float16)
    target = tf.reshape(target,
                        (output_length, num_targets))
    #print(target.shape)
    target = tf.slice(target,[crop_size,0],
                             [output_length - 2*crop_size,-1])

    bin_unique = tf.io.parse_tensor(example['bin_unique'],
                                  out_type=tf.int32)
    
    tss_mask = tf.io.parse_tensor(example['tss_mask'],
                                  out_type=tf.int32)
    tss_mask = tf.slice(tss_mask,[crop_size,0],
                             [output_length - 2*crop_size,-1])

    gene_name= tf.io.parse_tensor(example['gene_name'],out_type=tf.int32)
    gene_name = tf.tile(tf.expand_dims(gene_name,axis=0),[638])
    cell_types = tf.range(0,638)
    

    return {'sequence': tf.ensure_shape(sequence,
                                        [input_length,4]),
            'target': tf.ensure_shape(target,
                                      [output_length - 2*crop_size,num_targets]),
            'tss_mask': tf.ensure_shape(tss_mask,
                                        [output_length - 2*crop_size,1]),
            'gene_name': tf.ensure_shape(gene_name,
                                         [638,]),
            'bin_unique': bin_unique,
            'cell_types': tf.ensure_shape(cell_types,
                                           [638,])}

In [3]:
dataset = tf.data.TFRecordDataset("gs://genformer_data/expanded_originals_tss_mask_no_TF/393k/human/tfrecords/train-0-100.tfr",
                                  compression_type='ZLIB',
                                  num_parallel_reads=4)
#dataset = dataset.with_options(options)

dataset = dataset.map(lambda record: deserialize_val_TSS(record,
                                                         393216, 
                                                        896,
                                                        0,
                                                         10,
                                                         3387),
                      deterministic=False,
                      num_parallel_calls=4)


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


2023-02-27 15:01:59.924312: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-27 15:01:59.924397: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-27 15:01:59.924422: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tpu-genformer-v2-6): /proc/driver/nvidia/version does not exist
2023-02-27 15:01:59.924831: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [37]:
test = iter(dataset)

In [49]:
test = tf.ones(10,dtype=tf.float32)
tf.reshape(test, [-1,2])

<tf.Tensor: shape=(5, 2), dtype=float32, numpy=
array([[1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.],
       [1., 1.]], dtype=float32)>

In [45]:
test = tf.nn.experimental.stateless_dropout(test, 
                                     rate=0.20, 
                                     seed=[0,4]) / (1. / (1.0-0.20))
test = tf.expand_dims(test,axis=1)

In [48]:
tf.reshape(tf.tile(test, [1,5]),[-1])

<tf.Tensor: shape=(25,), dtype=float32, numpy=
array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 0., 0., 0., 0., 0.], dtype=float32)>

In [11]:
def one_hot(sequence):
    '''
    convert input string tensor to one hot encoded
    will replace all N character with 0 0 0 0
    '''
    vocabulary = tf.constant(['A', 'C', 'G', 'T'])
    mapping = tf.constant([0, 1, 2, 3])

    init = tf.lookup.KeyValueTensorInitializer(keys=vocabulary,
                                               values=mapping)
    table = tf.lookup.StaticHashTable(init, default_value=0)

    input_characters = tfs.upper(tfs.unicode_split(sequence, 'UTF-8'))

    out = tf.one_hot(table.lookup(input_characters), 
                      depth = 4, 
                      dtype=tf.float32)
    return out

def deserialize_val_TSS(serialized_example,input_length,max_shift,output_length_ATAC,
                        output_length,crop_size,output_res,predict_masked_atac_bool,atac_mask_dropout, use_global_acc, use_atac, log_atac,g):
    """Deserialize bytes stored in TFRecordFile."""
    feature_map = {
        'sequence': tf.io.FixedLenFeature([], tf.string),
        'atac': tf.io.FixedLenFeature([], tf.string),
        'cage': tf.io.FixedLenFeature([], tf.string),
        'tss_tokens': tf.io.FixedLenFeature([], tf.string),
        'cell_specific_conv_arr': tf.io.FixedLenFeature([], tf.string)
    }
    
    seq_shift = 5
    input_seq_length = input_length + max_shift

    ### rev_comp
    #rev_comp = random.randrange(0,2)

    data = tf.io.parse_example(serialized_example, feature_map)
    sequence = one_hot(tf.strings.substr(data['sequence'],
                                 seq_shift,input_length))
    
    
    #### parse ATAC and transform as specified
    atac = tf.ensure_shape(tf.io.parse_tensor(data['atac'],
                                              out_type=tf.float32),
                           [output_length_ATAC,1])
    diff = tf.math.sqrt(tf.nn.relu(atac - 64.0 * tf.ones(atac.shape)))
    atac = tf.clip_by_value(atac, clip_value_min=0.0, clip_value_max=64.0) + diff
    atac = tf.cast(tf.cast(atac,dtype=tf.float16),dtype=tf.float32) ### round to be consistent with Enformer
    atac_target = atac

    if log_atac: 
        atac = tf.math.log1p(atac)
        
        
    if not use_atac:
        atac = tf.math.abs(g.normal(atac.shape,
                             mean=0.0,
                             stddev=0.00001,
                             dtype=tf.float32))
                           
    ### here we generate a masked output vector length since we are predicting at 1536
    atac_mask = tf.ones(output_length // 2,dtype=tf.float32)
    atac_mask=tf.nn.experimental.stateless_dropout(atac_mask, 
                                                     rate=(atac_mask_dropout), 
                                                     seed=[0,seq_shift]) / (1. / (1.0-(atac_mask_dropout)))
    atac_mask = tf.expand_dims(atac_mask,axis=1)
    atac_mask = tf.tile(atac_mask, [1,2])
    atac_mask = tf.reshape(atac_mask, [-1])
    atac_mask = tf.expand_dims(atac_mask,axis=1)
    if atac_mask_dropout == 0.0:
        atac_mask_store = tf.ones_like(atac_mask)
    else:
        atac_mask_store = 1.0 - atac_mask ## invert the mask, since we want to store which values were masked and loss should be computed over
    atac_mask_store = tf.slice(atac_mask_store,
                            [crop_size,0],
                            [output_length-2*crop_size,-1])
    tiling_req = output_length_ATAC // output_length
    atac_mask = tf.expand_dims(tf.reshape(tf.tile(atac_mask, [1,tiling_req]),[-1]),axis=1)

    masked_atac = atac * atac_mask
    
    cage = tf.ensure_shape(tf.io.parse_tensor(data['cage'],
                                              out_type=tf.float32),
                           [output_length - 2*crop_size,1])
    diff = tf.math.sqrt(tf.nn.relu(cage - 850.0 * tf.ones(cage.shape)))
    cage = tf.clip_by_value(cage, clip_value_min=0.0, clip_value_max=850.0) + diff
    cage = tf.cast(tf.cast(cage,dtype=tf.float16),dtype=tf.float32) ### round to be consistent with Enformer
    
    tss_tokens = tf.io.parse_tensor(data['tss_tokens'],
                                  out_type=tf.int32)
    tss_tokens = tf.expand_dims(tss_tokens,axis=1)
    
    global_acc = tf.ensure_shape(tf.io.parse_tensor(data['cell_specific_conv_arr'],
                                              out_type=tf.float32),
                           [1536])
    global_acc=tf.expand_dims(global_acc,axis=0)
    global_acc = tf.math.asinh(global_acc)
    global_acc = (global_acc - tf.math.reduce_mean(global_acc)) / tf.math.reduce_std(global_acc)
    
    if not use_global_acc:
        global_acc = g.normal(global_acc.shape,
                              mean=0.0,
                              stddev=0.00025,
                              dtype=tf.float32)

    if predict_masked_atac_bool:
        atac_out = tf.reduce_sum(tf.reshape(atac_target, [-1,tiling_req]),axis=1,keepdims=True)
        diff = tf.math.sqrt(tf.nn.relu(atac_out - 64.0 * tf.ones(atac_out.shape)))
        atac_out = tf.clip_by_value(atac_out, clip_value_min=0.0, clip_value_max=64.0) + diff
        atac_out = tf.cast(tf.cast(atac_out,dtype=tf.float16),dtype=tf.float32) ### round to be consistent with Enformer
        atac_out = tf.slice(atac_out,
                            [crop_size,0],
                            [output_length-2*crop_size,-1])
        target = tf.concat([atac_out,cage],axis=1)
    

    print(tss_tokens.shape)
    
    if predict_masked_atac_bool:
        return {'sequence': tf.ensure_shape(sequence,
                                            [input_length,4]),
                'atac': tf.ensure_shape(masked_atac,
                                          [output_length_ATAC,1]),
                'atac_mask': tf.ensure_shape(atac_mask_store,
                                          [output_length-crop_size*2,1]),
                'target': tf.ensure_shape(target,
                                          [output_length-crop_size*2,2]),
                'tss_tokens': tf.ensure_shape(tss_tokens,
                                          [output_length-crop_size*2,1]),
                'global_acc': tf.ensure_shape(global_acc,
                                          [1,1536])}
    else:
        return {'sequence': tf.ensure_shape(sequence,
                                            [input_length,4]),
                'atac': tf.ensure_shape(atac,
                                          [output_length_ATAC,1]),
                'target': tf.ensure_shape(cage,
                                          [output_length-crop_size*2,1]),
                'tss_tokens': tf.ensure_shape(tss_tokens,
                                          [output_length-crop_size*2,1]),
                'global_acc': tf.ensure_shape(global_acc,
                                          [1,1536])}

In [12]:
dataset = tf.data.TFRecordDataset("gs://picard-testing-176520/test/train/HG_A375.tfr",
                                  compression_type='ZLIB',
                                  num_parallel_reads=4)
g = tf.random.Generator.from_non_deterministic_state()
dataset = dataset.map(lambda record: deserialize_val_TSS(record,
                                                         65536,
                                                         10,
                                                         16384,
                                                         512,
                                                         100,
                                                         128,
                                                         True,
                                                         0.0, 
                                                         True, 
                                                         True, 
                                                         True,
                                                         g),
                      deterministic=False,
                      num_parallel_calls=4)
test = iter(dataset)
out = next(test)

Exception ignored in: <function Executor.__del__ at 0x7fb68c713cb0>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/eager/executor.py", line 46, in __del__
    self.wait()
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/eager/executor.py", line 65, in wait
    pywrap_tfe.TFE_ExecutorWaitForAllPendingNodes(self._handle)
tensorflow.python.framework.errors_impl.InvalidArgumentError: Shape of tensor ExpandDims_3 [512,1] is not compatible with expected shape [312,1].
	 [[{{node EnsureShape_7}}]]
Additional GRPC error information from remote target /job:worker/replica:0/task:0:
:{"created":"@1678464977.178263258","description":"Error received from peer ipv4:10.93.119.234:8470","file":"external/com_github_grpc_grpc/src/core/lib/surface/call.cc","file_line":1056,"grpc_message":"Shape of tensor ExpandDims_3 [512,1] is not compatible with expected shape [312,1].\n\t [[{{node EnsureShape_7}}]]","grpc_status":3}


<unknown>


InvalidArgumentError: Shape of tensor ExpandDims_3 [512,1] is not compatible with expected shape [312,1].
	 [[{{node EnsureShape_7}}]]
Additional GRPC error information from remote target /job:worker/replica:0/task:0:
:{"created":"@1678465122.916985199","description":"Error received from peer ipv4:10.93.119.234:8470","file":"external/com_github_grpc_grpc/src/core/lib/surface/call.cc","file_line":1056,"grpc_message":"Shape of tensor ExpandDims_3 [512,1] is not compatible with expected shape [312,1].\n\t [[{{node EnsureShape_7}}]]","grpc_status":3}

In [ ]:
out = next(test)
out

In [ ]:
total = 0
counter = 0
for k in range(10000):
    out = g.uniform([], 0, 3,dtype=tf.int32)
    if out == 0:
        counter += 1
    total += 1

{'sequence': <tf.Tensor: shape=(65536, 4), dtype=float32, numpy=
 array([[0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        ...,
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.]], dtype=float32)>,
 'atac': <tf.Tensor: shape=(16384, 1), dtype=float32, numpy=
 array([[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [0.6374239 ],
        [0.20122541],
        [0.5118993 ]], dtype=float32)>,
 'atac_mask': <tf.Tensor: shape=(312, 1), dtype=float32, numpy=
 array([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]